Color Dominant Extraction

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import os
import csv
import matplotlib.pyplot as plt

# Function to extract dominant colors from an image
def extract_dominant_colors(image_path, k=5):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pixels = image.reshape(-1, 3)

    # KMeans clustering to find k dominant colors
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(pixels)

    # Get the cluster centers (dominant colors) and their labels
    dominant_colors = kmeans.cluster_centers_.astype(int)
    labels = kmeans.labels_

    # Count the number of pixels assigned to each dominant color
    label_counts = np.bincount(labels)

    # Calculate the percentage of each color
    total_pixels = len(labels)
    color_percentages = [(label_counts[i] / total_pixels) * 100 for i in range(k)]

    # Rank the colors by percentage (highest to least)
    ranked_colors = sorted(zip(dominant_colors, color_percentages), key=lambda x: -x[1])

    return ranked_colors

# Function to display the image beside its dominant colors
def display_thumbnail_with_palette(image_path, dominant_colors, title):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Create a color palette
    palette = np.zeros((image.shape[0], 300, 3), dtype='uint8')
    step = 300 // len(dominant_colors)
    for i, (color, _) in enumerate(dominant_colors):
        palette[:, i * step:(i + 1) * step] = color
    
    # Combine image and palette horizontally
    combined = np.hstack((image, palette))
    
    plt.figure(figsize=(10, 5))
    plt.imshow(combined)
    plt.title(title)
    plt.axis('off')
    plt.show()

# Set the path to the folder containing the images
image_folder = r'C:\Users\yesha\Desktop\College\4th year\1st Semester\THS-ST1\thumbnail_extract\thumbnails'  # Replace with your folder path

# Get a list of image file names in the folder and limit to 2938
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))][:2938]

# Create a CSV file and write headers
with open('Color_Dominant.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Top 5 Dominant Colors', 'Top 5 Dominant Colors Percentage'])

    # Process each image and extract top 5 dominant colors
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        dominant_colors = extract_dominant_colors(image_path, k=5)

        # Prepare the top 5 dominant colors and their percentages
        colors_list = [color.tolist() for color, _ in dominant_colors]
        percentages_list = [(f"{color.tolist()} , {round(percentage, 2)}%") for color, percentage in dominant_colors]

        # Write the image data into the CSV
        writer.writerow([image_file, colors_list, ', '.join(percentages_list)])

        # Display the image beside its dominant colors
        display_thumbnail_with_palette(image_path, dominant_colors, f"Top 5 Dominant Colors for {image_file}")

print("Data has been saved to 'Color_Dominant.csv'.")
